# baseline v3

이 베이스라인 코드는 `사전학습 모델 로드`, `배치 학습`, `파인튜닝`, `양자화`, `PEFT` 등이 적용된 버전입니다.

Colab의 GPU 환경에서 개발되었습니다.
- 런타임 - 런타임 유형 변경 - GPU로 변경(T4 GPU 등)



# 환경 준비

개발 환경에 필요한 라이브러리 버전을 고정하고 최신 버전으로 라이브러리를 업데이트합니다.

- 아래 셀 실행
- 실행 완료 후 런타임 - 세션 다시 시작

In [1]:
# !kaggle competitions download -c 2025-ssafy-14 

In [2]:
%pip -q install "transformers>=4.44.2" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" datasets pillow pandas torch torchvision --upgrade

Note: you may need to restart the kernel to use updated packages.


# 데이터 준비

개발에 필요한 데이터를 준비합니다.

- train.csv, train 폴더
- test.csv, test 폴더
- sample_submission.csv

본 베이스라인은 colab에서 구글 드라이브를 마운트하여 사용합니다.

데이터를 압축 해제하는데 몇 분 정도의 시간이 소요됩니다.

#### 실습 참고 내용

    챕터 2-2 합성 데이터 실습
    - 구글 드라이브 마운트 : drive()

In [3]:
# # 압축 해제
# !unzip "2025-ssafy-14.zip" -d "/content/"

# 라이브러리, 데이터, 설정

In [4]:
import os, re, math, random
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Dict, List, Any
from transformers import (
    AutoModelForVision2Seq,
    AutoProcessor,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 사전 학습 모델 정의
MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
IMAGE_SIZE = 384
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
train_df = pd.read_csv("/content/train.csv")
test_df  = pd.read_csv("/content/test.csv")

# 학습데이터 200개만 추출
#train_df = train_df.sample(n=200, random_state=SEED).reset_index(drop=True)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu128 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


Device: cuda


In [5]:
print(train_df)
#print(test_df)

              id                  path                          question  \
0     train_0001  train/train_0001.jpg       이 사진 속 운동기구가 설치된 장소는 어디일까요?   
1     train_0002  train/train_0002.jpg       이 사진에 보이는 전통 한국 건축물은 무엇인가요?   
2     train_0003  train/train_0003.jpg             이 사진에서 보이는 탈것은 무엇인가요?   
3     train_0004  train/train_0004.jpg              이 사진에서 보이는 새는 무엇인가요?   
4     train_0005  train/train_0005.jpg   이 사진에서 사람들이 모여서 보고 있는 것은 무엇인가요?   
...          ...                   ...                               ...   
3882  train_3883  train/train_3883.jpg               이 사진이 찍힌 도시는 어디일까요?   
3883  train_3884  train/train_3884.jpg       이 사진에 보이는 신호등의 불빛은 어떤 색인가요?   
3884  train_3885  train/train_3885.jpg             사진 속 고양이는 어디에 누워 있나요?   
3885  train_3886  train/train_3886.jpg  이 사진에서 볼 수 있는 구조물의 주요 특징은 무엇인가요?   
3886  train_3887  train/train_3887.jpg                   사진 속 음식은 무엇인가요?   

                   a           b             c          d answer  
0             학교 운동장

# 모델, Processor

7.5GB 정도의 모델 다운로드가 진행됩니다. 10~20분 정도가 소요됩니다.

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - LoRA 구현 : LoraConfig()

In [6]:
# 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 프로세서
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)

# 사전학습 모델
base_model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# 양자화 모델로 로드
base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

# LoRA 세팅
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)

# PEFT 모델 생성
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
/opt/conda/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.12s/it]


trainable params: 18,576,384 || all params: 3,773,199,360 || trainable%: 0.4923


# 프롬프트 템플릿

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - 프롬프트 템플릿 : convert_to_chatml(), formatting_prompts_func()

In [7]:
# 모델 지시사항
SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)

# 프롬프트
def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
    )

# Custom Dataset, Collator

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - TensorDataset()

    챕터 5-2 데이터 생성 및 파인튜닝 (향후 학습 분량)
    - IntentDataset()

In [8]:
# 커스텀 데이터셋
class VQAMCDataset(Dataset):
    def __init__(self, df, processor, train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.train = train

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        q = str(row["question"])
        a, b, c, d = str(row["a"]), str(row["b"]), str(row["c"]), str(row["d"])
        user_text = build_mc_prompt(q, a, b, c, d)

        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
            {"role":"user","content":[
                {"type":"image","image":img},
                {"type":"text","text":user_text}
            ]}
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            messages.append({"role":"assistant","content":[{"type":"text","text":gold}]})

        return {"messages": messages, "image": img}

# 데이터 콜레이터
@dataclass
class DataCollator:
    processor: Any
    train: bool = True

    def __call__(self, batch):
        texts, images = [], []
        for sample in batch:
            messages = sample["messages"]
            img = sample["image"]

            text = self.processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
            images.append(img)

        enc = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt"
        )

        if self.train:
            enc["labels"] = enc["input_ids"].clone()

        return enc


# DataLoader

#### 실습 참고 내용

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 데이터로더 정의 : DataLoader()

In [9]:
#하이퍼 파라미터 설정
EPOCHS = 2
LEARNING_RATE = 1e-5
GRADIENT_ACCUMULATION_STEPS = 4
WARMUP_RATIO = 0.03
BATCH_SIZE = 4
WEIGHT_DECAY = 0.01

# --- 환경 및 경로 설정 ---
# Mixed Precision에 사용할 데이터 타입 (사용하지 않으려면 None)
DTYPE = torch.bfloat16 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SAVE_DIR = "/content/qwen2_5_vl_3b_lora"

In [10]:
# 검증용 데이터 분리
split = int(len(train_df)*0.8)
valid_split = int(len(train_df)*0.5)
train_subset, valid_subset = train_df.iloc[:split], train_df.iloc[split:]
valid_subset, test_subset = valid_subset.iloc[:valid_split], valid_subset.iloc[valid_split:] 
# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# 데이터로더
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0)

print(f"Train samples: {len(train_ds)}, Valid samples: {len(valid_ds)}")

Train samples: 3109, Valid samples: 778


# fine-tuning

- 200개만 학습 : 10~20분 소요

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - 모델 정의 : SimpleMLP(), SequentialMLP()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [ ]:
import torch
import math
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

# --- 학습 환경 초기화 ---
model = model.to(DEVICE)

# 옵티마이저 및 스케줄러 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
num_training_steps = EPOCHS * math.ceil(len(train_loader) / GRADIENT_ACCUMULATION_STEPS)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(num_training_steps * WARMUP_RATIO),
    num_training_steps=num_training_steps
)

# Mixed Precision 스케일러
scaler = torch.cuda.amp.GradScaler(enabled=(DTYPE is not None))

# --- 학습 루프 시작 ---
# --- 학습 루프 시작 ---
global_step = 0
latest_val_loss = float("inf")
for epoch in range(EPOCHS):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch")
    
    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        
        # --- 그래디언트 계산 ---
        with torch.cuda.amp.autocast(dtype=DTYPE):
            outputs = model(**batch)
            loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
        
        scaler.scale(loss).backward()

        # --- 정해진 스텝마다 파라미터 업데이트 ---
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            # 옵티마이저 스텝
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

            # --- 주기적인 검증 ---
            if global_step > 0 and global_step % 10 == 0:
                model.eval()
                val_loss = 0.0
                with torch.no_grad(), torch.cuda.amp.autocast(dtype=DTYPE):
                    for vb in valid_loader:
                        vb = {k: v.to(DEVICE) for k, v in vb.items()}
                        val_loss += model(**vb).loss.item()
                
                latest_val_loss = val_loss / len(valid_loader)
                model.train() # 다시 학습 모드로 전환
            
            # 진행률 표시줄 업데이트 (loss는 근사치)
            progress_bar.set_postfix({
                "train_loss": f"{loss.item() * GRADIENT_ACCUMULATION_STEPS:.3f}",
                "val_loss": f"{latest_val_loss:.3f}"
            })

# (에포크의 마지막에 남은 그래디언트 처리 - 선택 사항이지만 권장)
if len(train_loader) % GRADIENT_ACCUMULATION_STEPS != 0:
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)
    scheduler.step()
    global_step += 1

# --- 모델 저장 ---
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print(f"✅ Saved model and processor to: {SAVE_DIR}")

/tmp/ipykernel_81604/3188553988.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DTYPE is not None))
Epoch 1/2:   0%|          | 0/778 [00:00<?, ?batch/s]/tmp/ipykernel_81604/3188553988.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=DTYPE):
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
Epoch 1/2:   5%|▌         | 39/778 [06:08<4:18:54, 21.02s/batch, train_loss=11.797, val_loss=inf]/tmp/ipykernel_81604/3188553988.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` 

# inference

30분~1시간 소요

#### 실습 참고 내용

    챕터4-1 RAG 기반 Customer Service AI 에이전트 개발
    - 데이터 파서 : langchain_core.output_parsers(), StrOutputParser()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [ ]:
test_df = test_subset
print (test_df)

Empty DataFrame
Columns: [id, path, question, a, b, c, d, answer]
Index: []


In [ ]:
# 데이터 파서 : 모델의 응답에서 선지를 추출
from sklearn.metrics import accuracy_score # 정확도 계산을 위해 import

def extract_choice(text: str) -> str:
    text = text.strip().lower()

    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a"
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last

    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a"
# 추론을 위해 모든 레이어 활성화
model.eval()
preds = []
labels = [] # 정답을 저장할 리스트

# 추론 루프
for i in tqdm(range(len(test_df)), desc="Inference", unit="sample"):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])
    label = row["answer"]
    labels.append(label)

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {"role":"user","content":[
            {"type":"image","image":img},
            {"type":"text","text":user_text}
        ]}
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

    with torch.no_grad():
        out_ids = model.generate(**inputs, max_new_tokens=2, do_sample=False,
                                 eos_token_id=processor.tokenizer.eos_token_id)
    
    # 새로 생성된 부분만 디코딩 (더 안정적인 방법)
    input_len = inputs['input_ids'].shape[1]
    generated_ids = out_ids[:, input_len:]
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    preds.append(extract_choice(output_text))

# --- ✅ 정확도 측정 코드 추가 ---
accuracy = accuracy_score(labels, preds)
print(f"✅ Accuracy: {accuracy * 100:.2f}%")
# -----------------------------

# 제출 파일 생성
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("/content/submission.csv", index=False)
print("Saved /content/submission.csv")

Inference: 0sample [00:00, ?sample/s]

Inference: 0sample [00:00, ?sample/s]

✅ Accuracy: nan%
Saved /content/submission.csv



/opt/conda/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:552: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/opt/conda/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# 모델 응답 예시
for txt in preds:
    print(txt)

a
a
a
d
c
c
c
a
a
a
